In [52]:
import pandas as pd
import numpy as np

df = pd.read_csv('test_data1.csv', sep=' ')
df

,任务编号,紧前任务,任务时间
0,1,NaN,5
1,3,1,2
2,2,NaN,4
3,5,"2,3",6
4,6,3,3
5,7,"5,6",2
6,8,"5,7",5
7,4,2,1
8,9,"4,8",2


In [59]:
df['任务时间'] = df['任务时间'].astype(str).str.extract('(\d+)').astype(float)
def process_predecessors(pred_str):
    if pd.isna(pred_str):
        return []
    else:
        return [int(x.strip()) for x in str(pred_str).split(',')]

predecessors = {task: process_predecessors(pred) for task, pred in zip(df['任务编号'], df['紧前任务'])}
successors = {task: [] for task in df['任务编号']}
for task, preds in predecessors.items():
    for pred in preds:
        successors[pred].append(task)

task_times = dict(zip(df['任务编号'], df['任务时间']))

 
def get_rpw(task_times, successors):
    all_successors = {task: [] for task in task_times.keys()}
    rpw_values = {}
    for task in task_times:
        if not successors[task]: 
            rpw_values[task] = task_times[task] 
    while len(rpw_values) < len(task_times):
        for task in task_times:
            if task not in rpw_values:  
                if all(succ in rpw_values for succ in successors[task]):
                    prev_all_successors = [all_successors[succ] for succ in successors[task]]
                    # print(prev_all_successors, successors[task])
                    all_successors[task] = list(set(np.concatenate((*prev_all_successors, successors[task]))))
                    # print(all_successors[task])
                    rpw_values[task] = task_times[task] + sum(task_times[succ] for succ in all_successors[task])
                    # print(f"task {task}, rpw {rpw_values[task]}")
                    del task
    return rpw_values
rpw_values = get_rpw(task_times, successors)
df['RPW'] = df['任务编号'].map(rpw_values)

df

,任务编号,紧前任务,任务时间,RPW
0,1,NaN,5.0,25.0
1,3,1,2.0,20.0
2,2,NaN,4.0,20.0
3,5,"2,3",6.0,15.0
4,6,3,3.0,12.0
5,7,"5,6",2.0,9.0
6,8,"5,7",5.0,7.0
7,4,2,1.0,3.0
8,9,"4,8",2.0,2.0


In [63]:
predecessors

{1: [],
 3: [1],
 2: [],
 5: [2, 3],
 6: [3],
 7: [5, 6],
 8: [5, 7],
 4: [2],
 9: [4, 8]}

In [64]:
def allocate_workstations(df, predecessors, cycle_time=10):
    workstations = {1: []}  
    station_times = {1: 0}  
    allocated_tasks = set()
    
    sorted_tasks = df.sort_values('RPW', ascending=False)
    
    current_station = 1
    while len(allocated_tasks) < len(df):
        tasks_allocated_this_round = False
        
        for _, task in sorted_tasks.iterrows():
            task_id = task['任务编号']
            if task_id in allocated_tasks:
                continue
                
            cur_predecessors = predecessors[task_id]
            if not all(pred in allocated_tasks for pred in cur_predecessors):
                continue
                
            if station_times[current_station] + task['任务时间'] <= cycle_time:
                workstations[current_station].append(task_id)
                station_times[current_station] += task['任务时间']
                allocated_tasks.add(task_id)
                tasks_allocated_this_round = True
        
        if not tasks_allocated_this_round:
            current_station += 1
            workstations[current_station] = []
            station_times[current_station] = 0
            
    return workstations

# Use the function
workstation_assignments = allocate_workstations(df, predecessors)
for station, tasks in workstation_assignments.items():
    print(f"Workstation {station}: Tasks {tasks}")

Workstation 1: Tasks [1, 3, 6]
Workstation 2: Tasks [2, 5]
Workstation 3: Tasks [7, 8, 4, 9]


In [55]:
successors

{1: [3],
 3: [5, 6],
 2: [5, 4],
 5: [7, 8],
 6: [7],
 7: [8],
 8: [9],
 4: [9],
 9: []}